# Model Serving Function

In [2]:
import mlrun

In [4]:
%nuclio cmd -c pip install lightgbm

In [5]:
%nuclio config spec.build.baseImage = "mlrun/mlrun"
%nuclio config kind = "serving"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'
%nuclio: setting kind to 'serving'


In [7]:
import numpy as np
from cloudpickle import load

class LGBMModel(mlrun.serving.V2ModelServer):
    
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body):
        try:
            feats = np.asarray(body['inputs'])
            result = self.model.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [8]:
# nuclio: end-code

## Deploy and Test The Function

In [9]:
models_path = 'https://s3.wasabisys.com/iguazio/models/lightgbm/SampleModel.pkl'

In [10]:
fn = mlrun.code_to_function('NYC-taxi-serving', description="NYC taxi fare prediction",
                     categories=['serving', 'ml'],
                     labels={'author': 'edmondg', 'framework': 'lightgbm'},
                     code_output='.')
fn.spec.default_class = 'LGBMModel'

In [11]:
fn.add_model('nyc-taxi-server', model_path=models_path)

In [12]:
# deploy the function
fn.apply(mlrun.mount_v3io())
address = fn.deploy()

> 2020-11-15 07:48:55,421 [info] deploy started
[nuclio] 2020-11-15 07:48:57,561 (info) Build complete
[nuclio] 2020-11-15 07:49:10,775 (info) Function deploy complete
[nuclio] 2020-11-15 07:49:10,787 done creating default-nyc-taxi-serving, function address: 192.168.224.70:31186
> 2020-11-15 07:49:10,800 [warning] warning!, server (0.5.3-rc1) and client (unstable) ver dont match


In [13]:
# test the function
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 3, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2]]}'''
fn.invoke('/v2/models/nyc-taxi-server/predict', my_data)

{'id': 'a2000662-fb7c-4df9-bbee-dbe406805ee8',
 'model_name': 'nyc-taxi-server',
 'outputs': [25.374309065093435]}